## Importar datos de Netsuite

Este notebook sirve para conectarse e importar datos del ERP de Oracle "Netsuite".

La diferencia más significativa en el código es el driver que se utiliza en el import con JDBC

In [ ]:
%run DEV/Utils/LakehouseFunctions  

In [ ]:
# Buscamos los orígenes que contengan NETSUITE
df_netsuite = pd.read_sql(f"SELECT * FROM Origins WHERE Name = 'NETSUITE'", engine_conn)

# Obtenemos el Endpoint y el nombre de la BBDD de Netsuite
endpoint = df_netsuite["Endpoint"][0]
database = df_netsuite["Name"][0]

# Obtenemos el Usuario y la contraseña
username = dbutils.secrets.get(keyvault_name, df_netsuite["Username"][0]) 
secret = dbutils.secrets.get(keyvault_name, df_netsuite["Secret"][0])

# Especificamos el driver de JDBC para obtener datos de Netsuite
driver ="com.netsuite.jdbc.openaccess.OpenAccessDriver"

#Consulta al listado de tablas añadidas en el catálogo para su descarga
df_datasets = pd.read_sql("SELECT Datasets.Id as DatasetId, Datasets.Name as DatasetName, Query, Keys, ChangeTrackingVersion, TableName FROM Origins LEFT JOIN Datasets on Origins.Id = Datasets.OriginId WHERE Origins.Name = 'NETSUITE' and IsActive = 1", engine_conn)
df_datasets

In [ ]:
Incremental=False

# Por cada uno de los datasets, importamos la tabla que contiene Netsuite
for _, table in df_datasets.iterrows():
    table_name = table['TableName']
    keys=table['Keys']
    table_df = spark.read.format ( "jdbc" ) \
        .option ( "driver" , driver) \
        .option ( "url" , endpoint) \
        .option ("user", username) \
        .option ("password", secret) \
        .option ( "dbtable" , table_name) \
        .load ()
    
    # Borramos el raw data de las tablas de Netsuite subidas anteriormente en el esquema RAW
    delete_raw_data("Netsuite_"+table_name)

    # Guardamos los datos nuevos en el esquema RAW
    write_raw_data(table_df,"Netsuite_"+table_name)

    # Pasamos a la capa final los datos de RAW acabados de importar
    write_standardized_data("Netsuite_"+table['TableName'], Incremental, keys)